In [1]:
import pandas as pd
from sklearn import preprocessing

In [2]:
postulaciones = pd.read_csv('Sources/postulaciones_for_training.csv').drop('Unnamed: 0', axis=1)

In [3]:
postulantes = pd.read_csv('Sources/postulantes_proc.csv').drop('Unnamed: 0', axis=1)
avisos = pd.read_csv('Sources/fiuba_6_avisos_detalle_limpio.csv').drop('Unnamed: 0', axis=1)
test = pd.read_csv('Sources/test_final_100k.csv')
preds = pd.DataFrame()

In [4]:
postulaciones = postulaciones.merge(postulantes, on='idpostulante')
postulaciones = postulaciones.merge(avisos, on='idaviso')
resultados = postulaciones['sepostulo']
postulaciones.drop(['titulo','descripcion','idaviso','idpostulante','sepostulo'],axis=1, inplace=True)
postulaciones.head()

,fechanacimiento,sexo,nivel,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,FEM,MASC,NO_DECLARA,TendenciaSexo,prom_edad
0,33.0,MASC,11,Full-time,Senior / Semi-Senior,Administración,Adecco -Región Office,186,664,682,17,NaN,27.071164
1,25.0,FEM,11,Full-time,Senior / Semi-Senior,Administración,Adecco -Región Office,186,664,682,17,NaN,27.071164
2,19.0,FEM,3,Full-time,Senior / Semi-Senior,Administración,Adecco -Región Office,186,664,682,17,NaN,27.071164
3,35.0,FEM,12,Full-time,Senior / Semi-Senior,Administración,Adecco -Región Office,186,664,682,17,NaN,27.071164
4,20.0,MASC,11,Full-time,Senior / Semi-Senior,Administración,Adecco -Región Office,186,664,682,17,NaN,27.071164


In [5]:
test = test.merge(postulantes, on='idpostulante')
test = test.merge(avisos, on='idaviso')
test.drop(['titulo','descripcion','idaviso','idpostulante','id'],axis=1, inplace=True)
test.head()

,fechanacimiento,sexo,nivel,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,FEM,MASC,NO_DECLARA,TendenciaSexo,prom_edad
0,38.0,MASC,10,Full-time,Otro,Administración,TODO EMPRESARIAL Y LABORAL S.A.,20,840,701,14,NaN,27.319715
1,25.0,FEM,6,Full-time,Otro,Administración,TODO EMPRESARIAL Y LABORAL S.A.,20,840,701,14,NaN,27.319715
2,38.0,MASC,12,Full-time,Otro,Administración,TODO EMPRESARIAL Y LABORAL S.A.,20,840,701,14,NaN,27.319715
3,22.0,FEM,11,Full-time,Otro,Administración,TODO EMPRESARIAL Y LABORAL S.A.,20,840,701,14,NaN,27.319715
4,19.0,MASC,3,Full-time,Otro,Administración,TODO EMPRESARIAL Y LABORAL S.A.,20,840,701,14,NaN,27.319715


In [6]:
test = pd.concat([test,postulaciones])

In [7]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for col in test.columns:
    if not postulaciones[col].dtype == 'O': continue
    test[col] = test[col].fillna('na')
    le.fit(test[col])
    test[col] = le.transform(test[col])

In [8]:
postulaciones = test[100000:].fillna(-1)
test = test[:100000].fillna(-1)

In [9]:
partitions = []
partitions_res = []
n = 3
index = int(postulaciones.shape[0]/n)
for i in range(n):
    partitions.append(postulaciones[index*i:index*(i+1)])
    partitions_res.append(resultados[index*i:index*(i+1)])

In [10]:
aviso = None
postulantes = None
import xgboost as xgb
# read in data
dtrain = xgb.DMatrix(data=partitions[0],label=partitions_res[0])
dtest = xgb.DMatrix(data=partitions[2])
# specify parameters via map
param = {'max_depth':50, 'eta':1, 'silent':1, 'objective':'multi:softmax','num_class':3, 'subsample':1}
num_round = 50
bst = xgb.train(param, dtrain, num_round)
# make prediction
preds['xg'] = bst.predict(dtest)

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
#X_train, X_test, y_train, y_test = train_test_split(postulaciones, resultado, test_size=0.2)
clf=RandomForestClassifier(n_estimators=50,n_jobs=-1,warm_start=True,criterion='entropy')
clf.fit(partitions[1],partitions_res[1])
preds['ran']=clf.predict(partitions[2])

In [12]:
from sklearn.naive_bayes import BernoulliNB
nb = BernoulliNB()
nb.fit(preds,partitions_res[2])
BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)


ValueError: Expected input with 2 features, got 13 instead

In [ ]:
import xgboost as xgb
# read in data
dtrain = xgb.DMatrix(predicciones,label=resultados)
# specify parameters via map
param = {'max_depth':50, 'eta':1, 'silent':1, 'objective':'multi:softmax','num_class':3, 'subsample':1}
num_round = 50
bst = xgb.train(param, dtrain, num_round)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
#X_train, X_test, y_train, y_test = train_test_split(postulaciones, resultado, test_size=0.2)
clf=RandomForestClassifier(n_estimators=50,n_jobs=-1,warm_start=True,criterion='entropy')
clf.fit(postulaciones,resultados)

In [19]:
preds = pd.DataFrame()
dtest = xgb.DMatrix(data=test)
preds['xg'] = bst.predict(dtest)
preds['ran']=clf.predict(test)
pred = nb.predict(preds)

ValueError: Expected input with 2 features, got 13 instead

In [ ]:
pred.to_csv('Sources/resultados_pip.csv')